# HW3 Lexical Chain
# Shruti Gupta (A20381966) and Sagar Mane(A20379756)

In [25]:
# Load require packages 
# import nltk
# nltk.download('wordnet')
# nltk.download('wordnet')
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')

# Text Preprocessing

In [26]:
'''
preprocess

Function for preprocessing text file
by removing number, stopword, punctuation 
and non alaphanumeric character

Parameter:
filename: Name of the read file

Return:
List of words

'''

def preprocess(filename):
    stemmer = PorterStemmer()
    lemmatiser = WordNetLemmatizer()
    tokens = []
    words = []
    stop_words = set(stopwords.words('english'))
    for line in open(filename): 
#         print(line)
        all_words = line.split()
#         print(all_words)
        for w in all_words:
            w1 = w
            w = stemmer.stem(w)
#             w = lemmatiser.lemmatize(w)
            if not w.lower() in stop_words and not w in string.punctuation and w.isalpha():
                tokens.append(w)
                words.append(w1)
    return tokens

In [27]:
tokens = preprocess('smalltext.txt')
tokens

['girl',
 'women',
 'person',
 'hat',
 'skirt',
 'girl',
 'shirt',
 'skirt',
 'skirt',
 'girl',
 'women',
 'person',
 'shirt',
 'rome',
 'mumbai',
 'one',
 'two',
 'ten',
 'one',
 'ten',
 'three',
 'want',
 'told',
 'woman']

# 1. Create lexical chains based on the lexical relationships of synonymy, antonymy, and one level of hyper/hyponymy. In other words, if a noun is related by one of these relationships to an existing lexical chain, it should be added to it, otherwise it should start a new chain.

In [28]:
def getList(word):
    list_ = []
    w = wordnet.synsets(word)
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            list_.append(l.name())
            if l.antonyms():
                list_.append(l.antonyms()[0].name())

    for i in w[0].hyponyms():
        list_.append(i.lemmas()[0].name())
    return list_

In [29]:
'''
getChain
Function for finding lexical chain

Parameter:
tokens: Preprocess words list

Return:
List of lexical chain

'''

def getChain(tokens):
    list1 = tokens
    list2 = tokens
    list_ = []
    for i in range(len(list1)):
        list_.append([])

# print(counts)
    i=0
    for word1 in list1:
        for word2 in list2:
            w1 = lemmatiser.lemmatize(word1)
            w2 = lemmatiser.lemmatize(word2)
            wordFromList1 = wordnet.synsets(w1)
            wordFromList2 = wordnet.synsets(w2)
            if wordFromList1 and wordFromList2: 
                wup_sim = wordFromList1[0].wup_similarity(wordFromList2[0])
                try:
                    jcn_sim = wordFromList1[0].jcn_similarity(wordFromList2[0], brown_ic)
                except:
                    pass
#                 print(jcn_sim)
                if(wup_sim is not None and wup_sim > 0.6 and jcn_sim > 0.08):
                    list_[i].append(word2)
#             print (wordFromList1[0],wordFromList2[0],s)
        i = i+1

# print((list_))

    new_k = []
    for elem in list_:
        if elem not in new_k and len(elem)>0:
            new_k.append(elem)
    return new_k

In [30]:
# Get Lexicon chain list
tokens = preprocess('smalltext.txt')
k=getChain(tokens)
counts = Counter(tokens)
n=1
for i in k:
    l = set(i)
    chain = 'Chain '+ str(n) + ': '
    for j in l:     
        chain = chain + j + "(" +  str(counts.get(j)) + "), "
    n = n + 1
    print(chain)

Chain 1: woman(1), person(2), girl(3), women(2), 
Chain 2: skirt(3), hat(1), shirt(2), 
Chain 3: mumbai(1), rome(1), 
Chain 4: two(1), three(1), ten(2), one(2), 
Chain 5: want(1), 
Chain 6: told(1), 


## Only allow a noun to enter a lexical chain if it is within a threshold number of words (this should be a parameter) to the closest element of the chain.

In [31]:
#Create noun list
nouns = {x.name().split('.', 1)[0] for x in wordnet.all_synsets('n')}

#Get lixicon chain list
tokens = preprocess('smalltext.txt')
k=getChain(tokens)
counts = Counter(tokens)

#Remove Non noun words
for i in k:
    for j in i:
        w1 = lemmatiser.lemmatize(j)
        if w1 not in nouns:
            i.remove(j)
          
n=1
for i in k:
    l = set(i)
    if len(l)>0:
        chain = 'Chain '+ str(n) + ': '
        for j in l: 
            chain = chain + j + "(" +  str(counts.get(j)) + "), "
        n = n + 1
        print(chain)


Chain 1: woman(1), person(2), girl(3), women(2), 
Chain 2: skirt(3), hat(1), shirt(2), 
Chain 3: mumbai(1), rome(1), 
Chain 4: two(1), three(1), ten(2), one(2), 


# Extra credit: Use the lexical chains to automatically create a summary of the input article.

In [32]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest
lemmatiser = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english') + list(punctuation))

'''
wordScore

Function for calculating score of chain

Parameter:
chains: Lexical chains of similar words
tokens: list of preprocess word in text file
word: name of word for identify lexical chain

Return:
Score of lexical chain

'''
def wordScore(chains,tokens,word):
    length = 0
    homogeneity = len(set(tokens))/len(tokens)
    for chain in chains:
        if word in chain:
            length = len(chain)
    rank = length * homogeneity   
   
    return rank
             
'''
summarize

Function for summarizing large text articles.

Parameter: 
text: Large text file
n: no. of senteces in summarize text

return:
List of summarize text

'''
def summarize(text, n):
    sents = sent_tokenize(text)
#     print(sents)
#     assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
#     print(word_sent)
    tokens = []
    for s in word_sent:
        for w in s:
            w = stemmer.stem(w)
            if not w.lower() in stop_words and not w in string.punctuation and w.isalpha():
                tokens.append(w)
    chains = getChain(tokens)

#COMPUTING RANKING FOR EACH WORD
    _freq = defaultdict(int)
    for s in word_sent:
        for word in s:
            if word not in stop_words:
                _freq[word] = wordScore(chains,tokens,word)

#COMPTING RANKING FOR EACH SENTENCE
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
        for w in sent:
            if w in _freq:
                ranking[i] += _freq[w]
#     print(ranking)
   
    sents_idx = nlargest(n, ranking, key=ranking.get)   
    return [sents[j] for j in sents_idx]


In [33]:
lines = ''
for line in open('text.txt'):
    lines += line
summarize(lines,5)

['It was Coach Snyder’s first year as our coach.',
 'One of the first conversations we had, he came up to me and told me he had watched every game France played in the World Cup.',
 'He remembered how we lost to Spain in the group stage — by 24 points — and he remembered how we faced them again in the quarterfinals and won.',
 'Spain had the Gasol brothers, Serge Ibaka, Ricky Rubio, a great team.',
 'I had just spent the summer playing in the World Cup for the French national team.']